# Scrapping and merging code
- This notebook gives the code used to scrap the target (the success rate at the 'bervet des collèges' from the site 'Le journal des Femmes'
- it also gives the merge procedure with a table with a lot of data per college from data_gouv. 
- Finally there is a small cleaning code for some macro socio economic data per cities
- More information about the data is provided in the starting_kit_notebook

In [ ]:
import pandas as pd
pd.set_option('display.max_columns', 500)
pd.set_option("display.max_rows", 500)
import re
import recordlinkage as rl

## Geographic data

- a table with some socio economic data per city

In [5]:
# donnees per city 

cities_data = pd.read_excel("./data//donnees_geographiques/insee_communes/base_cc_comparateur.xls", sheet_name = 0, header = 4, skiprows = 1)
arrondissements_data = pd.read_excel("./data/donnees_geographiques/insee_communes/base_cc_comparateur.xls", sheet_name = 1, header = 4, skiprows = 1)
#drop Paris, Lyon and Marseilles in cities_data
cities_data = cities_data.drop(cities_data[cities_data.LIBGEO.isin(["Paris", "Lyon", "Marseille"])].index)
cities_data = pd.concat([cities_data, arrondissements_data], ignore_index=True)


In [120]:
cities_data.head()

,CODGEO,LIBGEO,REG,DEP,P15_POP,P10_POP,SUPERF,NAIS1015,DECE1015,P15_MEN,NAISD17,DECESD17,P15_LOG,P15_RP,P15_RSECOCC,P15_LOGVAC,P15_RP_PROP,NBMENFISC15,PIMP15,MED15,TP6015,P15_EMPLT,P15_EMPLT_SAL,P10_EMPLT,P15_POP1564,P15_CHOM1564,P15_ACT1564,ETTOT15,ETAZ15,ETBE15,ETFZ15,ETGU15,ETGZ15,ETOQ15,ETTEF115,ETTEFP1015
0,01001,L'Abergement-Clémenciat,84,01,767.0,784.0,15.95,35.0,26.0,306.000000,8.0,2.0,345.367367,306.000000,14.997092,24.370275,260.000000,308.0,NaN,22228.000000,NaN,90.301043,57.321423,73.107018,463.000000,33.000000,376.000000,50.0,11.0,3.0,5.0,24.0,5.0,7.0,10.0,0.0
1,01002,L'Abergement-de-Varey,84,01,241.0,221.0,9.15,16.0,7.0,100.168724,4.0,2.0,165.080117,100.168724,49.638124,15.273269,85.292181,100.0,NaN,22883.333333,NaN,12.924205,4.990049,17.361344,142.814815,9.917695,121.987654,19.0,2.0,3.0,0.0,11.0,3.0,3.0,2.0,0.0
2,01004,Ambérieu-en-Bugey,84,01,14127.0,13835.0,24.60,1103.0,574.0,6250.799769,204.0,122.0,6962.512740,6250.799769,120.235153,591.477818,2898.975000,6216.0,53.312962,19735.200000,17.227132,7510.860164,6805.315072,7718.550230,8959.916801,1060.731602,6703.560420,1337.0,7.0,52.0,131.0,907.0,290.0,240.0,399.0,109.0
3,01005,Ambérieux-en-Dombes,84,01,1619.0,1616.0,15.92,113.0,44.0,618.000000,18.0,11.0,663.000000,618.000000,12.000000,33.000000,471.000000,624.0,NaN,23182.666667,NaN,292.598576,219.449212,297.907990,1038.000000,66.000000,838.000000,141.0,14.0,7.0,27.0,78.0,20.0,15.0,27.0,5.0
4,01006,Ambléon,84,01,109.0,116.0,5.88,8.0,5.0,52.518182,1.0,1.0,72.518182,52.518182,11.428571,8.571429,37.654545,NaN,NaN,NaN,NaN,5.945455,3.963636,5.168688,71.345455,7.927273,57.472727,7.0,0.0,0.0,0.0,5.0,1.0,2.0,0.0,0.0


In [6]:

#correspondance between the postal code and the insee code
cp_ci = pd.read_csv("./data/donnees_geographiques/donnees_postal_code/code-postal-code-insee-2015.csv", sep = ";")

In [7]:
cities_data.CODGEO = cities_data.CODGEO.astype(str) #codgeo = insee code
mapper_insee_cp = dict(zip(cp_ci.INSEE_COM, cp_ci.Code_postal.apply(lambda x: "nan" if x !=x else str(x)[:-2])))
cities_data["postal_code"] = cities_data.CODGEO.apply(lambda x: mapper_insee_cp[x] if x in mapper_insee_cp.keys() else "nan" )

cities_data.rename(columns = {"CODGEO":"insee_code", "P15_POP":"population","MED15":"med_std_living", "TP6015":"poverty_rate",
                              "P15_CHOM1564":"nb_unemployed", "P15_ACT1564":"labour_force"}, inplace=True)
cities_data['unemployment_rate'] = cities_data['nb_unemployed']/cities_data['labour_force']

In [123]:
cities_data.head()

,insee_code,LIBGEO,REG,DEP,population,P10_POP,SUPERF,NAIS1015,DECE1015,P15_MEN,NAISD17,DECESD17,P15_LOG,P15_RP,P15_RSECOCC,P15_LOGVAC,P15_RP_PROP,NBMENFISC15,PIMP15,med_std_living,poverty_rate,P15_EMPLT,P15_EMPLT_SAL,P10_EMPLT,P15_POP1564,nb_unemployed,labour_force,ETTOT15,ETAZ15,ETBE15,ETFZ15,ETGU15,ETGZ15,ETOQ15,ETTEF115,ETTEFP1015,postal_code
0,01001,L'Abergement-Clémenciat,84,01,767.0,784.0,15.95,35.0,26.0,306.000000,8.0,2.0,345.367367,306.000000,14.997092,24.370275,260.000000,308.0,NaN,22228.000000,NaN,90.301043,57.321423,73.107018,463.000000,33.000000,376.000000,50.0,11.0,3.0,5.0,24.0,5.0,7.0,10.0,0.0,1400
1,01002,L'Abergement-de-Varey,84,01,241.0,221.0,9.15,16.0,7.0,100.168724,4.0,2.0,165.080117,100.168724,49.638124,15.273269,85.292181,100.0,NaN,22883.333333,NaN,12.924205,4.990049,17.361344,142.814815,9.917695,121.987654,19.0,2.0,3.0,0.0,11.0,3.0,3.0,2.0,0.0,1640
2,01004,Ambérieu-en-Bugey,84,01,14127.0,13835.0,24.60,1103.0,574.0,6250.799769,204.0,122.0,6962.512740,6250.799769,120.235153,591.477818,2898.975000,6216.0,53.312962,19735.200000,17.227132,7510.860164,6805.315072,7718.550230,8959.916801,1060.731602,6703.560420,1337.0,7.0,52.0,131.0,907.0,290.0,240.0,399.0,109.0,1500
3,01005,Ambérieux-en-Dombes,84,01,1619.0,1616.0,15.92,113.0,44.0,618.000000,18.0,11.0,663.000000,618.000000,12.000000,33.000000,471.000000,624.0,NaN,23182.666667,NaN,292.598576,219.449212,297.907990,1038.000000,66.000000,838.000000,141.0,14.0,7.0,27.0,78.0,20.0,15.0,27.0,5.0,1330
4,01006,Ambléon,84,01,109.0,116.0,5.88,8.0,5.0,52.518182,1.0,1.0,72.518182,52.518182,11.428571,8.571429,37.654545,NaN,NaN,NaN,NaN,5.945455,3.963636,5.168688,71.345455,7.927273,57.472727,7.0,0.0,0.0,0.0,5.0,1.0,2.0,0.0,0.0,1300


In [8]:
cities_data.to_csv("./data/donnees_geographiques/cities_data.csv")

### Scrapping de la target : Taux moyen de reussite au brevet sur 3 ans 

In [2]:
from urllib.request import urlopen
from bs4 import BeautifulSoup

### Scrapping journal des femmes
- we scrapped the name of the college, its city and success rate
- WARNING : the code is quite long to run (one hour)
 

In [196]:
# code scrapping 
url = 'https://www.journaldesfemmes.fr/maman/ecole/classement/colleges/taux-reussite-brevet?'
html = urlopen(url)
soup = BeautifulSoup(html, "html")
last_page = 120

colleges = []
target = [] # the success rate
villes = []

for p in range(1, last_page +1):
    url2 = url+'page='+str(p)
    html2 = urlopen(url2)
    soup2 = BeautifulSoup(html2, "html")
    res = [i for i in soup2.find_all(['tr'])][1:] # not interested by the first row
    for i in res:
        #get the name
        colleges.append(i.find_all('td')[1].text)
        #get the target 
        target.append(i.find_all('td')[2].text)
        
        #get the city
        url_college = 'https://www.journaldesfemmes.fr'+ i.find('a').attrs['href']
        html_college = urlopen(url_college)
        soup_college = BeautifulSoup(html_college, 'html')
        villes.append(soup_college.find_all('tr')[3].find('a').text)
        assert len(colleges) == len(target)
        assert len(colleges) == len(villes)
        
        
       

In [197]:
data_colleges = pd.DataFrame({'Name':colleges,
                             'target':target,
                             'ville':villes})

In [201]:
def treat_rate(x):
    x = re.sub('\s','', x)
    x = re.sub('%','', x)
    x = re.sub(',','.', x)
    return np.float(x)
 
data_colleges['target'] = data_colleges['target'].apply(treat_rate)

In [159]:
def treat_college(x):
    x = re.sub('Collège ', '', x)
    return x


In [202]:
data_colleges.head()

,Name,target,ville
0,Collège privé de Marcq,100.0,Marcq-en-Barœul
1,Collège privé Saint-Charles,100.0,Saint-Pierre
2,Collège Saint-Thomas d'Aquin,100.0,Oullins
3,Collège Saint-Charles,100.0,Athis-Mons
4,Collège Passy Saint-Nicolas Buzenval,100.0,Rueil-Malmaison


In [203]:
#to csv 
data_colleges.to_csv('./data/college/target.csv')

## Merging

- We merge the target data frame with a table found on (source <a href= 'https://www.data.gouv.fr/fr/datasets/colleges-education-prioritaire/'> data.gouv </a> ) with a lot of informations about the college
- As a lot of colleges have the same name, we also need the city to do the merge.
- We use the clean function of the record linkage 

In [9]:
# target data
target = pd.read_csv('./data/college/target.csv', index_col=0)
target.head()

,Name,target,ville
0,Collège privé de Marcq,100.0,Marcq-en-Barœul
1,Collège privé Saint-Charles,100.0,Saint-Pierre
2,Collège Saint-Thomas d'Aquin,100.0,Oullins
3,Collège Saint-Charles,100.0,Athis-Mons
4,Collège Passy Saint-Nicolas Buzenval,100.0,Rueil-Malmaison


In [10]:
# data of data.gouv
rep_data = pd.read_csv("./data/college/fr-en-colleges-ep.csv", sep=';')
# we only select the 2017 year as this is the year of the ranking on the journal des femmes website
rep_data = rep_data[rep_data['Rentrée scolaire'] == 2017]
print('shape', rep_data.shape)
rep_data.rename(columns={'Patronyme':'Name', 'Commune nom':'City_name'}, inplace=True)
rep_data.head()


shape (5294, 59)


,Identifiant de l'établissement dans la base,Rentrée scolaire,Numéro établissement,Appartenance EP,Type établissement code,Type établissement nom,Name,Coordonnée X,Coordonnée Y,Secteur code,Secteur nom,Etablissement sensible,CATAEU2010,Situation relative à une zone rurale ou autre,Commune code,City_name,Commune et arrondissement code,Commune et arrondissement nom,Département code,Département nom,Académie code,Académie nom,Région code,Région nom,Région 2016 code,Région 2016 nom,Nb élèves,Nb 6èmes,Nb 5èmes,Nb 4èmes générales,Nb 3èmes générales,Nb 3èmes insertion,Nb dispositifs relais,Nb dispositifs initiation aux métiers en alternance,Nb 2nd cycle général ou technologique,Nb CAP ou BAC professionnel,Nb 6ème SEGPA,Nb 5ème SEGPA,Nb 4ème SEGPA,Nb 3ème SEGPA,Nb SEGPA,Nb 3èmes générales retardataires,Nb divisions,Nb 6èmes provenant d'une école EP,Nb 5èmes 4èmes et 3èmes générales Latin ou Grec,Nb 5èmes 4èmes et 3èmes générales,Nb élèves pratiquant langue rare,Nb 6èmes bilangues,Nb 6èmes 5èmes 4èmes et 3èmes générales sections européennes et internationales,Nb 6èmes 5èmes 4èmes et 3èmes générales,Nb 3émes générales et insertion rentrée précédente passés en 2nde GT,Nb 3émes générales et insertion rentrée précédente passés en cycle professionnel,Nb 3émes générales et insertion rentrée précédente,Longitude,Latitude,Inscription à DCOL,Inscription à DCOL Rs N-1,Situation relative à un QPV (quartier prioritaire de la Ville),Position
1964,20170560050A,2017.0,0560050A,HEP,21.0,COLLEGE,JULES SIMON,267972.0,6744464.8,1.0,PUBLIC,NON,111.0,urbain,56260,VANNES,56260,VANNES,56,MORBIHAN,14.0,RENNES,53,BRETAGNE,53,BRETAGNE,774.0,175.0,185.0,198.0,216.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,37.0,30.0,43.0,32.0,0.0,774.0,147.0,38.0,214.0,-2.760887,47.658668,NaN,-2.760887,47.658668,NaN,NaN,NaN,"47.6586681136,-2.76088651324"
1965,20170740929H,2017.0,0740929H,HEP,21.0,COLLEGE,RAOUL BLANCHARD,942287.5,6538643.7,1.0,PUBLIC,NON,111.0,urbain,74010,ANNECY,74010,ANNECY,74,HAUTE SAVOIE,8.0,GRENOBLE,82,RHONE-ALPES,84,AUVERGNE-ET-RHONE-ALPES,829.0,211.0,200.0,192.0,213.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,51.0,31.0,144.0,0.0,0.0,816.0,150.0,41.0,211.0,6.125997,45.904308,NaN,6.125997,45.904308,NaN,NaN,NaN,"45.9043076805,6.12599674762"
1966,20170900017E,2017.0,0900017E,HEP,21.0,COLLEGE,CHATEAUDUN,989168.7,6734317.0,1.0,PUBLIC,NON,111.0,urbain,90010,BELFORT,90010,BELFORT,90,TERRITOIRE DE BELFORT,3.0,BESANCON,43,FRANCHE-COMTE,27,BOURGOGNE-ET-FRANCHE-COMTE,344.0,79.0,83.0,86.0,84.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,15.0,16.0,0.0,0.0,0.0,332.0,54.0,22.0,96.0,6.853101,47.645728,NaN,6.853101,47.645728,NaN,NaN,NaN,"47.6457280427,6.85310105037"
1967,20170951696C,2017.0,0951696C,HEP,21.0,COLLEGE,GEORGES DUHAMEL,637692.1,6878029.4,1.0,PUBLIC,NON,111.0,urbain,95306,HERBLAY,95306,HERBLAY,95,VAL-D'OISE,25.0,VERSAILLES,11,ILE-DE-FRANCE,11,ILE-DE-FRANCE,372.0,102.0,85.0,84.0,85.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,20.0,14.0,0.0,12.0,0.0,356.0,57.0,20.0,84.0,2.148468,48.999161,NaN,2.148468,48.999161,NaN,NaN,NaN,"48.9991613692,2.14846829582"
1968,20170280803W,2017.0,0280803W,HEP,21.0,COLLEGE,EDOUARD HERRIOT,585352.4,6816567.4,1.0,PUBLIC,NON,111.0,urbain,28218,LUCE,28218,LUCE,28,EURE-ET-LOIR,18.0,ORLEANS-TOURS,24,CENTRE-VAL-DE-LOIRE,24,CENTRE-VAL-DE-LOIRE,517.0,124.0,131.0,133.0,116.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,16.0,20.0,0.0,23.0,0.0,504.0,76.0,29.0,119.0,1.449799,48.439256,NaN,1.449799,48.439256,NaN,NaN,NaN,"48.4392557302,1.44979875281"


In [11]:
from recordlinkage.preprocessing import clean

In [12]:
target.Name = clean(target.Name, strip_accents='unicode') # remove accent
target.Name = target.Name.apply(lambda x: re.sub('college ', '', x)) #remove the word college
target.Name = target.Name.apply(lambda x:re.sub('prive ', '', x)) #remove the word prive
target.ville = clean(target.ville, strip_accents='unicode')
rep_data.Name = clean(rep_data.Name)
rep_data.City_name = clean(rep_data.City_name)

In [13]:
target.head()

,Name,target,ville
0,de marcq,100.0,marcq en barul
1,saint charles,100.0,saint pierre
2,saint thomas daquin,100.0,oullins
3,saint charles,100.0,athis mons
4,passy saint nicolas buzenval,100.0,rueil malmaison


In [15]:
data_college = pd.merge(rep_data, target, left_on=['Name', 'City_name'], right_on=['Name', 'ville'], how='inner')
data_college.drop(columns=['ville'], axis=1, inplace=True) #drop the ville and keep city_name

In [16]:
data_college.head()

,Identifiant de l'établissement dans la base,Rentrée scolaire,Numéro établissement,Appartenance EP,Type établissement code,Type établissement nom,Name,Coordonnée X,Coordonnée Y,Secteur code,Secteur nom,Etablissement sensible,CATAEU2010,Situation relative à une zone rurale ou autre,Commune code,City_name,Commune et arrondissement code,Commune et arrondissement nom,Département code,Département nom,Académie code,Académie nom,Région code,Région nom,Région 2016 code,Région 2016 nom,Nb élèves,Nb 6èmes,Nb 5èmes,Nb 4èmes générales,Nb 3èmes générales,Nb 3èmes insertion,Nb dispositifs relais,Nb dispositifs initiation aux métiers en alternance,Nb 2nd cycle général ou technologique,Nb CAP ou BAC professionnel,Nb 6ème SEGPA,Nb 5ème SEGPA,Nb 4ème SEGPA,Nb 3ème SEGPA,Nb SEGPA,Nb 3èmes générales retardataires,Nb divisions,Nb 6èmes provenant d'une école EP,Nb 5èmes 4èmes et 3èmes générales Latin ou Grec,Nb 5èmes 4èmes et 3èmes générales,Nb élèves pratiquant langue rare,Nb 6èmes bilangues,Nb 6èmes 5èmes 4èmes et 3èmes générales sections européennes et internationales,Nb 6èmes 5èmes 4èmes et 3èmes générales,Nb 3émes générales et insertion rentrée précédente passés en 2nde GT,Nb 3émes générales et insertion rentrée précédente passés en cycle professionnel,Nb 3émes générales et insertion rentrée précédente,Longitude,Latitude,Inscription à DCOL,Inscription à DCOL Rs N-1,Situation relative à un QPV (quartier prioritaire de la Ville),Position,target
0,20170560050A,2017.0,0560050A,HEP,21.0,COLLEGE,jules simon,267972.0,6744464.8,1.0,PUBLIC,NON,111.0,urbain,56260,vannes,56260,VANNES,56,MORBIHAN,14.0,RENNES,53,BRETAGNE,53,BRETAGNE,774.0,175.0,185.0,198.0,216.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,37.0,30.0,43.0,32.0,0.0,774.0,147.0,38.0,214.0,-2.760887,47.658668,NaN,-2.760887,47.658668,NaN,NaN,NaN,"47.6586681136,-2.76088651324",95.6
1,20170740929H,2017.0,0740929H,HEP,21.0,COLLEGE,raoul blanchard,942287.5,6538643.7,1.0,PUBLIC,NON,111.0,urbain,74010,annecy,74010,ANNECY,74,HAUTE SAVOIE,8.0,GRENOBLE,82,RHONE-ALPES,84,AUVERGNE-ET-RHONE-ALPES,829.0,211.0,200.0,192.0,213.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,51.0,31.0,144.0,0.0,0.0,816.0,150.0,41.0,211.0,6.125997,45.904308,NaN,6.125997,45.904308,NaN,NaN,NaN,"45.9043076805,6.12599674762",88.9
2,20170951696C,2017.0,0951696C,HEP,21.0,COLLEGE,georges duhamel,637692.1,6878029.4,1.0,PUBLIC,NON,111.0,urbain,95306,herblay,95306,HERBLAY,95,VAL-D'OISE,25.0,VERSAILLES,11,ILE-DE-FRANCE,11,ILE-DE-FRANCE,372.0,102.0,85.0,84.0,85.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,20.0,14.0,0.0,12.0,0.0,356.0,57.0,20.0,84.0,2.148468,48.999161,NaN,2.148468,48.999161,NaN,NaN,NaN,"48.9991613692,2.14846829582",76.3
3,20170280803W,2017.0,0280803W,HEP,21.0,COLLEGE,edouard herriot,585352.4,6816567.4,1.0,PUBLIC,NON,111.0,urbain,28218,luce,28218,LUCE,28,EURE-ET-LOIR,18.0,ORLEANS-TOURS,24,CENTRE-VAL-DE-LOIRE,24,CENTRE-VAL-DE-LOIRE,517.0,124.0,131.0,133.0,116.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,16.0,20.0,0.0,23.0,0.0,504.0,76.0,29.0,119.0,1.449799,48.439256,NaN,1.449799,48.439256,NaN,NaN,NaN,"48.4392557302,1.44979875281",89.2
4,20170330129L,2017.0,0330129L,HEP,21.0,COLLEGE,henri dheurle,372116.5,6400997.6,1.0,PUBLIC,NON,111.0,urbain,33529,la teste de buch,33529,LA TESTE-DE-BUCH,33,GIRONDE,4.0,BORDEAUX,72,AQUITAINE,75,NOUVELLE-AQUITAINE,777.0,194.0,194.0,174.0,201.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,24.0,27.0,1.0,29.0,0.0,763.0,111.0,40.0,189.0,-1.135637,44.630780,NaN,-1.135637,44.630780,NaN,NaN,NaN,"44.6307801744,-1.13563658848",87.4


In [17]:
data_college.shape

(4186, 60)

In [18]:
drop_col = ["Identifiant de l'établissement dans la base",
            'Rentrée scolaire',
           'Numéro établissement',
            'Type établissement code',
           'Type établissement nom',
            'Secteur code', #all public
            'Secteur nom'
           ]

data_college.drop(columns=drop_col, axis=1, inplace=True)

In [19]:
data_college.to_csv('./data/college/data_college.csv') #merge df 